<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT20/blob/leoenge/Lab-1/leoenge_matrix_multiplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab: Matrix multiplication**
**Leo Enge**

# **Abstract**

In this lab, functions for calculating some of the standard vector and matrix operations were defined. It was done by first defining a very general function for calculating the matrix product upon which all the other calculations were defined. The accuracy of the functions were tested against the corresponding numpy functions and in all cases equaled to at least seven decimal places.

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [0]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019 Leo Enge (leoe@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

In [0]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

import unittest

# **Introduction**

In this lab, functions for calculating a set of standard vector and matrix operations are defined. The approach taken in this report is to define a very general function to calculate the matrix multiplication, upon which all the other calculations can then be defined, directly or indirectly. 



# **Methods**

## Matrix multiplication
We define a function which returns the matrix multiplication $(c_{i,j})$ of two matrices $(a_{i,j})$ and $(b_{i,j})$, by iterativley doing the computation
\begin{equation}
c_{i,j} = \sum_k a_{i,k}b_{k,j}
\end{equation}

The two matrices can be of any shapes as long as the number of columns of the first one equals the number of rows of the second one. That this is satisfied is checked before any computation is done.


In [0]:
def matrix_product(M1, M2):
  if not (M1.ndim == 2 and M2.ndim == 2):
    raise TypeError('Matrices must be two-dimensional.')
  if not (M1.shape[1] == M2.shape[0]):
    raise TypeError('The dimensions of the matrices are not compatible for multiplication.')
  result_matrix = np.zeros((M1.shape[0], M2.shape[1]))
  for i in range(0, result_matrix.shape[0]):
    for j in range(0, result_matrix.shape[1]):
      sum = 0
      for k in range(0, M1.shape[1]):
        sum += M1[i,k]*M2[k,j]
      result_matrix[i,j] = sum
  return result_matrix

### Testing the matrix multiplication
We first test that the expected errors are raised we run the function with a couple of different matrix dimensions and sizes that should not be admitted.

Then we test the accuracy of the matrix multiplication. Assuming the numpy function as the correct answer of the matrix multiplication (this is not completely right, but probably sufficiently so) we test wheter our matrix multiplication fails to be correct within some different decimal points for a set of random matrices with random shapes. We do so by using the numpy testing funtion *assert_almost_equal*.



In [0]:
class Test(unittest.TestCase):
  
  def test_dimension_and_size(self):
    M1 = np.array([1,1,1])
    M2 = np.array([2,2,2])
    M3 = np.array([[3,3,3],[3,3,3],[3,3,3]])
    M4 = np.array([[2,2],[2,2]])
    with self.assertRaises(TypeError):
      matrix_product(M1,M2)
    with self.assertRaises(TypeError):
      matrix_product(M1, M3)
    with self.assertRaises(TypeError):
      matrix_product(M3, M4)
  
  def test_accuray(self):
    for _ in range(0,100):
      dim = np.random.randint(1,10)
      M1 = np.random.rand(np.random.randint(1,10), dim)
      M2 = np.random.rand(dim, np.random.randint(1,10))
      for i in range(3,7):
        np.testing.assert_almost_equal(matrix_product(M1, M2), np.matmul(M1, M2), decimal=i)

if __name__ == '__main__':
  unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.100s

OK


## Matrix-vector multiplication
A vector is a matrix with only one row or one column. Hence the vector matrix multiplication can be done using the already defined method for matrix multiplication if the vector is expressed as a matrix of the right shape.

Before this is done, the function checks that the dimensions and sizes are compatible for this multiplication.**bold text**

In [0]:
def matrix_vector_product(matrix, vector):
  if not vector.ndim == 1:
    raise TypeError('Vector must be one-dimensional')
  if not vector.size == matrix.shape[1]:
    raise TypeError('The size of the vector and the dimension of the matrix are not compatible for muliplication.')
  return np.squeeze(matrix_product(matrix, np.array([vector]).transpose()))

### Testing the matrix-vector multiplication
Just as before we can test that the function raises *TypeError* when the dimensions and sizes are wrong, but here we only test the accuracy of the muliplication against the corresponding numpy function.

In [0]:
class Test(unittest.TestCase):
  
  def test_accuracy(self):
    for _ in range(0,100):
      dim = np.random.randint(1,10)
      M = np.random.rand(np.random.randint(1,10), dim)
      v = np.random.rand(dim)
      for i in range(3,7):
        np.testing.assert_almost_equal(matrix_vector_product(M, v), M.dot(v), decimal=i)


if __name__ == '__main__':
  unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.059s

OK


## Scalar multiplication
The scalar product can be calculated by
\begin{equation}
\langle v, u \rangle = v^T u
\end{equation}
so we calculate the scalar product using this identiy and the previously defined function for matrix multiplication.

In [0]:
def scalar_product(v1, v2):
  if not (v1.ndim == 1 and v2.ndim == 1):
    raise TypeError('Vectors must be one-dimensional.')
  if not v1.size == v2.size:
    raise TypeError('Vectors must be of the same length.')
  return matrix_product(np.array([v1]), np.array([v2]).transpose())[0]

### Testing the scalar product
We test the accuray of the scalar product against the corresponding numpy function for a set of random vectors of random sizes.

In [0]:
class Test(unittest.TestCase):
  
  def test_accuracy(self):
    for _ in range(0,100):
      dim = np.random.randint(1,10)
      v1 = np.random.rand(dim)
      v2 = np.random.rand(dim)
      for i in range(3,7):
        np.testing.assert_almost_equal(scalar_product(v1, v2), v1.dot(v2), decimal=i)


if __name__ == '__main__':
  unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.053s

OK


## Norm and distance
The Euclidean norm $\|v\|$ can be defined as 
\begin{equation}
\|v\|^2 = \langle v, v \rangle 
\end{equation}
so we calculate the norm using the function for the scalar product previously defined. The Euclidean disctance $\|u-v\|$ can now be calculated using this function for the Euclidean norm.

The accuracy of these two functions is then tested against the corresponding numpy function in the same manner as before for a set of random vectors of random sizes. Since the accuracy of the distance only depends on the accuracy of the norm, this is not tested explicitly.


In [0]:
def norm(v):
  if not v.ndim == 1:
    raise TypeError('Vector must be one-dimensional.')
  return np.sqrt(scalar_product(v,v))

In [0]:
def distance(x,y):
  if not (x.ndim == 1 and y.ndim == 1):
    raise TypeError('Vectors must be one-dimensional.')
  if not x.size == y.size:
    raise TypeError('Vectors must be of the same length.')
  return(norm(x-y))

In [0]:
class Test(unittest.TestCase):
  
  def test_norm_accuracy(self):
    for _ in range(0,100):
      dim = np.random.randint(1,10)
      v = np.random.rand(dim)
      for i in range(3,7):
        np.testing.assert_almost_equal(norm(v), np.linalg.norm(v), decimal=i)

if __name__ == '__main__':
  unittest.main(argv=['first-arg-is-ignored'], exit=False)


.
----------------------------------------------------------------------
Ran 1 test in 0.061s

OK


# **Results**

All the functions defined were tested with a set of 100 ranom matrices or vectors of size 10 or smaller. And the result of all the function were equal to their corresponding numpy function to at least 7 decimal places.

# **Discussion**

Assuming that numpy's functions for calculating the operations in this lab are very close to the true value, which is reasonably to assume since it is a a well-known, widley used a well tested library for numeric calculations, we got the result that the very simple functions defined in this lab are correct to at least seven decimal points. This is not however not unexpected since the operations themselves are quite simple and do not lend themselves to much ambiguity in what computations should be made.

Instead of using the approach of defining a very general function for matrix multiplication upon which the rest of the calculations are based, as was done in this report, one can instead begin by defining the scalar multiplication, then build the matrix-vector and matrix-matrix multiplication upon this one. That approach could be favourable since reading the code might give a more easy-to-understand and intutive idea of what the multiplications accutually consist of. Whereas the method used now feels more like a algorithmic manipulation of indicies. In terms of what computations are acctually done, and when, it makes no difference though.